In [29]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import re

In [30]:
# Charger les données JSON
json_file_path = "linked_files.json"
with open(json_file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)

In [31]:
def extract_technique_section(content):
    # Rechercher la section Technique jusqu'au prochain titre de niveau 1
    match = re.search(r"# Technique\s*(.*?)(?=\n# |\Z)", content, re.DOTALL)
    if match:
        # Extraire le contenu
        technique_text = match.group(1).strip()
        # Supprimer toutes les lignes commençant par un '#' (titres et sous-titres)
        cleaned_text = re.sub(r"^#.*", "", technique_text, flags=re.MULTILINE)
        # Nettoyer les espaces et les lignes vides
        return "\n".join(line.strip() for line in cleaned_text.splitlines() if line.strip())
    return ""


In [36]:
# Extraire les titres et le contenu technique
documents = []
for entry in json_data:
    title = entry["title"]
    content = entry["content"]
    # Extraire uniquement la partie "Technique" du contenu
    technique_section = extract_technique_section(content)
    # Combiner le titre et le contenu technique
    combined_text = f"{title} {technique_section}"
    documents.append(combined_text)

print(documents[18])

Réduction des pertes de chaleur par les ouvertures du four Identifier les ouvertures du fours et réduire les pertes de chaleur associées.
Les déperditions de chaleur par rayonnement peuvent se produire par les ouvertures des fours servant au chargement/déchargement. Ceci est particulièrement important dans les fours fonctionnant à une température supérieure à 500 °C.
Les ouvertures sont, entre autres, les trappes d’évacuation et les cheminées, les trous de regard permettant de vérifier visuellement le procédé, les portes laissées partiellement ouvertes pour s’adapter à une tâche surdimensionnée, le chargement et le déchargement des matières et/ou des combustibles, etc.


In [37]:
# Initialiser le modèle SBERT et le Cross-Encoder
sbert_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-electra-base")
#"antoinelouis/crossencoder-electra-base-french-mmarcoFR"
#cross-encoder/ms-marco-MiniLM-L-12-v2
#cross-encoder/ms-marco-MiniLM-L-6-v2
#cross-encoder/camembert-base
#cross-encoder/ms-marco-electra-base


# Calculer les embeddings des documents
embeddings = sbert_model.encode(documents)

# Indexer avec FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
faiss.normalize_L2(embeddings)
index.add(embeddings)

In [38]:
# Fonction de recherche
def search(query, top_k=10):
    # Vectoriser la requête
    query_embedding = sbert_model.encode([query])
    faiss.normalize_L2(query_embedding)
    
    # Rechercher les documents les plus proches avec FAISS
    _, faiss_results = index.search(query_embedding, top_k)
    faiss_scores = [(documents[i], json_data[i]) for i in faiss_results[0]]
    
    # Réordonner les résultats avec le Cross-Encoder
    cross_scores = cross_encoder.predict([(query, doc[0]) for doc in faiss_scores])
    ranked_results = [x for _, x in sorted(zip(cross_scores, faiss_scores), reverse=True)]
    
    return ranked_results[:top_k]

In [41]:
# Exemple de recherche
query = "Haute pression"
results = search(query)
for doc, metadata in results:
    print(f"Titre: {metadata['title']}")
    print(f"Code: {metadata['identifier']}")
    print("---")

Titre: Pompe de surpression pour des hautes pressions ponctuelles
Code: 64
---
Titre: Réduction de la pression quand cela est possible
Code: 204
---
Titre: Réduction de la pression de l'air comprimé au minimum requis
Code: 332
---
Titre: Pompe à chaleur haute température pour valoriser la vapeur basse pression
Code: 104
---
Titre: Optimisation de la pression des soufflettes
Code: 158
---
Titre: Echangeurs de pression
Code: 1060
---
Titre: Surpresseur et réseau d'air multi pressions pour répondre aux besoins localisés
Code: 143
---
Titre: Compresseur basse pression à vis
Code: 971
---
Titre: Niveau de pression optimum pour les pompes à vide
Code: 945
---
Titre: Réglage des températures de consigne et des débits d'extraction
Code: 1523
---
